In [1]:
#import dependencies

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import requests

In [2]:
#convert the dta file to csv

data = pd.io.stata.read_stata('relationship_data.dta')
data.to_csv('data.csv')


In [3]:
# Reference the file where the CSV is located
csv_path = "relationship_data_file.csv"

# Import the data into a Pandas DataFrame
relationship_data_df = pd.read_csv(csv_path)
relationship_data_df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (48,103) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,CaseID,CASEID_NEW,qflag,weight1,weight1_freqwt,weight2,weight1a,weight1a_freqwt,weight_combo,...,hcm2017q24_met_through_family,hcm2017q24_met_through_friend,hcm2017q24_met_through_as_nghbrs,hcm2017q24_met_as_through_cowork,w6_subject_race,interracial_5cat,partner_mother_yrsed,subject_mother_yrsed,partner_yrsed,subject_yrsed
0,0,2,2014039,Qualified,NaN,NaN,0.8945,NaN,NaN,0.277188,...,no,no,no,no,White,no,12.0,14.0,12.0,14.0
1,1,3,2019003,Qualified,0.9078,71115.0,NaN,0.9026,70707.0,1.020621,...,no,no,no,yes,White,no,12.0,16.0,17.0,17.0
2,2,5,2145527,Qualified,0.7205,56442.0,NaN,0.7164,56121.0,0.810074,...,no,no,no,no,White,no,9.0,7.5,14.0,17.0
3,3,6,2648857,Qualified,1.2597,98682.0,1.3507,1.2524,98110.0,0.418556,...,no,no,no,no,White,no,16.0,12.0,12.0,12.0
4,4,7,2623465,Qualified,0.8686,68044.0,NaN,0.8636,67652.0,0.976522,...,no,no,yes,no,White,no,14.0,17.0,16.0,16.0


In [4]:
#drop all NaN values; reset index
cleaned_new_df = new_df.dropna().reset_index()

cleaned_new_df.head()

NameError: name 'new_df' is not defined

In [5]:
#pull the columns i will need to do the transformations into my own table

employment_age_cols = ["CASEID_NEW",
            "age_when_met",
            "Q21A_Year",
            "Q21D_Year",
            "ppmarit",
            "ppeduc",
            "ppethm"
           ]

employment_age_df = relationship_data_df[employment_age_cols].copy()

cleaned_employment_df = employment_age_df.dropna()

cleaned_employment_df.head()

,CASEID_NEW,age_when_met,Q21A_Year,Q21D_Year,ppmarit,ppeduc,ppethm
1,2019003,21.0,1983,1996,Married,Masters degree,"White, Non-Hispanic"
2,2145527,36.0,2006,2008,Married,Masters degree,"White, Non-Hispanic"
4,2623465,25.0,1983,1984,Married,Bachelors degree,"White, Non-Hispanic"
5,2917189,23.0,1981,1982,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic"
6,2919423,15.0,1966,1969,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic"


In [6]:
# remove all "refused" values from the marriage (Q21D_Year) year

indexNames = cleaned_employment_df[cleaned_employment_df['Q21D_Year'] == "Refused"].index

print(indexNames)

employment_df = cleaned_employment_df.drop(indexNames)

employment_df.head()


Int64Index([ 196,  351,  501,  535,  827, 1035, 1103, 1185, 1275, 1403, 1418,
            2048, 2238, 2864, 2892, 2984, 2985, 3286, 3294],
           dtype='int64')


,CASEID_NEW,age_when_met,Q21A_Year,Q21D_Year,ppmarit,ppeduc,ppethm
1,2019003,21.0,1983,1996,Married,Masters degree,"White, Non-Hispanic"
2,2145527,36.0,2006,2008,Married,Masters degree,"White, Non-Hispanic"
4,2623465,25.0,1983,1984,Married,Bachelors degree,"White, Non-Hispanic"
5,2917189,23.0,1981,1982,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic"
6,2919423,15.0,1966,1969,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic"


In [7]:
newIndexNames = employment_df[employment_df['Q21A_Year'] > employment_df['Q21D_Year']].index

final_employment_df = employment_df.drop(newIndexNames)

final_employment_df.head()

,CASEID_NEW,age_when_met,Q21A_Year,Q21D_Year,ppmarit,ppeduc,ppethm
1,2019003,21.0,1983,1996,Married,Masters degree,"White, Non-Hispanic"
2,2145527,36.0,2006,2008,Married,Masters degree,"White, Non-Hispanic"
4,2623465,25.0,1983,1984,Married,Bachelors degree,"White, Non-Hispanic"
5,2917189,23.0,1981,1982,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic"
6,2919423,15.0,1966,1969,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic"


In [8]:
final_employment_df['time_till_marriage'] = final_employment_df['Q21D_Year'].astype(int) - final_employment_df['Q21A_Year'].astype(int)

final_employment_df['marriage_age'] = final_employment_df['time_till_marriage'] + final_employment_df['age_when_met']

final_employment_df.head()



,CASEID_NEW,age_when_met,Q21A_Year,Q21D_Year,ppmarit,ppeduc,ppethm,time_till_marriage,marriage_age
1,2019003,21.0,1983,1996,Married,Masters degree,"White, Non-Hispanic",13,34.0
2,2145527,36.0,2006,2008,Married,Masters degree,"White, Non-Hispanic",2,38.0
4,2623465,25.0,1983,1984,Married,Bachelors degree,"White, Non-Hispanic",1,26.0
5,2917189,23.0,1981,1982,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic",1,24.0
6,2919423,15.0,1966,1969,Married,HIGH SCHOOL GRADUATE - high school DIPLOMA or ...,"White, Non-Hispanic",3,18.0


In [9]:
final_employment_df['ppeduc'].unique()

array(['Masters degree', 'Bachelors degree',
       'HIGH SCHOOL GRADUATE - high school DIPLOMA or the equivalent (GED)',
       'Associate degree', 'Professional or Doctorate degree',
       'Some college, no degree', '12th grade NO DIPLOMA', '9th grade',
       '11th grade', '10th grade', '7th or 8th grade', '5th or 6th grade',
       'No formal education', '1st, 2nd, 3rd, or 4th grade'], dtype=object)

In [10]:
#establish conneciton to MySQL

db_url = {
   'database': 'relationship_data_db',
   'drivername': 'mysql',
   'username': 'root',
   'password': 'Outlaw1!',
   'host': '127.0.0.1',
   'query': {'charset': 'utf8'},  # the key-point setting
}

engine = create_engine(URL(**db_url), encoding='utf8')

In [ ]:
#save my df into a MySQL table
final_employment_df.to_sql(name='final_employment_df', con=engine, if_exists='append', index=True)